In [ ]:
import pandas as pd
import numpy as np
import scipy.stats

In [ ]:
data = pd.read_csv('https://github.com/Enzuigirii/Main/raw/main/my-notebooks/data/ab_data.zip')
data.head()

,user_id,timestamp,group,landing_page,converted
0,851104,2017-01-21 22:11:48.556739,control,old_page,0
1,804228,2017-01-12 08:01:45.159739,control,old_page,0
2,661590,2017-01-11 16:55:06.154213,treatment,new_page,0
3,853541,2017-01-08 18:28:03.143765,treatment,new_page,0
4,864975,2017-01-21 01:52:26.210827,control,old_page,1


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 294478 entries, 0 to 294477
Data columns (total 5 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   user_id       294478 non-null  int64 
 1   timestamp     294478 non-null  object
 2   group         294478 non-null  object
 3   landing_page  294478 non-null  object
 4   converted     294478 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 11.2+ MB


In [ ]:
data.group.unique()

array(['control', 'treatment'], dtype=object)

In [ ]:
pivot_table = pd.pivot_table(data, values='user_id', index='landing_page', columns='group', aggfunc=np.count_nonzero)
pivot_table

group,control,treatment
landing_page,,
new_page,1928,145311
old_page,145274,1965


In [ ]:
drop = data[((data.landing_page == 'new_page') & (data.group == 'control')) | ((data.landing_page == 'old_page') & (data.group == 'treatment'))]
data = data.drop(drop.index)
data.shape

(290585, 5)

In [ ]:
pivot_table = pd.pivot_table(data, values='user_id', index='landing_page', columns='group', aggfunc=np.count_nonzero)
pivot_table

group,control,treatment
landing_page,,
new_page,NaN,145311.0
old_page,145274.0,NaN


In [ ]:
duplicated = data.user_id[data.user_id.duplicated() == True]
data[data.user_id.isin(duplicated)]

,user_id,timestamp,group,landing_page,converted
1899,773192,2017-01-09 05:37:58.781806,treatment,new_page,0
2893,773192,2017-01-14 02:55:59.590927,treatment,new_page,0


In [ ]:
data = data.drop_duplicates(subset='user_id', keep='first')
data.shape

(290584, 5)

In [ ]:
data.groupby(by='group')['converted'].agg([np.mean, np.std])

,mean,std
group,,
control,0.120386,0.325414
treatment,0.118808,0.323564


Судя по статистике, новый дизайн, работал немного хуже. 

Примем за нулевую гипотезу, что вероятность конверсии нового и старого дизайна не отличаются. Уровень достоверности выберем 95% (α = 0.05)

In [ ]:
data.groupby(by=['converted', 'group', 'landing_page']).count()

user_id  timestamp
converted group     landing_page                    
0         control   old_page       127785     127785
          treatment new_page       128046     128046
1         control   old_page        17489      17489
          treatment new_page        17264      17264

In [ ]:
control = data[data.group == 'control'] 
treatment = data[data.group == 'treatment']

control_click = control.converted.sum() 
control_noclick = control.converted.size - control.converted.sum() 
treatment_click = treatment.converted.sum() 
treatment_noclick = treatment.converted.size - treatment.converted.sum()

data = np.array([[control_noclick, control_click], [treatment_noclick, treatment_click]])
data

array([[127785,  17489],
       [128046,  17264]])

# Chi-Squared Test

In [ ]:
dof = data.size - sum(data.shape) + data.ndim - 1
dof

1

In [ ]:
total = data.sum()

expected_values = np.array([[(data[0].sum() * (data[0][0] + data[1][0]) / total), 
                             (data[0].sum() * (data[0][1] + data[1][1]) / total)], 
                            [(data[1].sum() * (data[0][0] + data[1][0]) / total), 
                             (data[1].sum() * (data[0][1] + data[1][1]) / total)]])

chi_squared = ((data[0][0] - expected_values[0][0])**2 / expected_values[0][0]) \
               + ((data[0][1] - expected_values[0][1])**2 / expected_values[0][1]) \
               + ((data[1][0] - expected_values[1][0])**2 / expected_values[1][0]) \
               + ((data[1][1] - expected_values[1][1])**2 / expected_values[1][1])
chi_squared

1.7185222540121272

In [ ]:
chi, p = scipy.stats.chi2_contingency(data, correction = False)[:2]

print('chi statistic:', chi)
print('p-value:', p)

chi statistic: 1.7185222540121272
p-value: 0.18988337448194853


Предположив уровень значимости в 5%, мы не можем отклонить нулевую гипотезу, поскольку p > α. Следовательно нет значительной разницы между стырам и новым дизайном.

In [ ]:
control_ctr = control_click / (control_click + control_noclick) 
treatment_ctr = treatment_click / (treatment_click + treatment_noclick) 
print(control_ctr, treatment_ctr)


0.1203863045004612 0.11880806551510564


При ручной проверке так же заметно, что нет сильной разницы

# Z-test

In [ ]:
from statsmodels.stats.proportion import proportions_ztest, proportion_confint


n_con = control.converted.count()
n_treat = treatment.converted.count()

successes = [control_click, treatment_click]
nobs = [n_con, n_treat]

z_stat, p_value = proportions_ztest(successes, nobs=nobs)
(lower_con, lower_treat), (upper_con, upper_treat) = proportion_confint(successes, nobs=nobs, alpha=0.05)

print('z statistic:', z_stat)
print('p-value:', p_value)
print('95% доверительный интервал для контрольной группы: [{:.4f}, {:.4f}]'.format(lower_con, upper_con))
print('95% доверительный интервал для эксперементальной группы: [{:.4f}, {:.4f}]'.format(lower_treat, upper_treat))

z statistic: 1.3109241984234394
p-value: 0.18988337448195103
95% доверительный интервал для контрольной группы: [0.1187, 0.1221]
95% доверительный интервал для эксперементальной группы: [0.1171, 0.1205]


p-value также больше α. Не можем отклонить нулевую гипотезу. Кроме того, базовое значение коэфициента конверсии 11.8% входит в доверительный интервал для эксперементальной группы

# Байесовский подход

In [ ]:
from scipy.stats import beta
import numpy as np

from math import lgamma
from numba import jit

#defining the functions used
@jit
def h(a, b, c, d):
    num = lgamma(a + c) + lgamma(b + d) + lgamma(a + b) + lgamma(c + d)
    den = lgamma(a) + lgamma(b) + lgamma(c) + lgamma(d) + lgamma(a + b + c + d)
    return np.exp(num - den)

@jit
def g0(a, b, c):    
    return np.exp(lgamma(a + b) + lgamma(a + c) - (lgamma(a + b + c) + lgamma(a)))

@jit
def hiter(a, b, c, d):
    while d > 1:
        d -= 1
        yield h(a, b, c, d) / d

def g(a, b, c, d):
    return g0(a, b, c) + sum(hiter(a, b, c, d))

def calc_prob_between(beta1, beta2):
    return g(beta1.args[0], beta1.args[1], beta2.args[0], beta2.args[1])


n_con = control.converted.count()
n_treat = treatment.converted.count()

#Создаем две бета-функции для контрольной и эксперементальной группы
a_c, b_c = control_click+1, n_con-control_click+1
beta_c = beta(a_c, b_c)
a_t, b_t = treatment_click+1, n_treat-treatment_click+1
beta_t = beta(a_t, b_t)

#смотрим насколько выше конверсия для тестовой группы, чем для контрольной:
lift=(beta_t.mean()-beta_c.mean())/beta_c.mean()

#рассчитываем вероятность того, что эксперементальная группа лучше контрольной
prob=calc_prob_between(beta_t, beta_t)

print ('конверсия в эксперементальной группе изменится на {:.2%} с {:.2%} вероятностью'.format(lift, prob))

конверсия в эксперементальной группе изменится на -1.31% с 50.00% вероятностью
